# Loading Digits Dataset

In [1]:
import numpy as np
import sklearn
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

digits_data = load_digits()

X = digits_data.data
y = digits_data.target

X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.3,random_state=0)

# Linear Regression

In [2]:
def get_coeff_linear(X_train, y_train):
    w = np.linalg.pinv(X_train)
    w = np.dot(w, y_train)
    return w

def evaluate_err(X_train, X_test, y_train, y_test, w): 
    #### TO-DO #####
    p_train = np.dot(X_train, w)
    train_error = np.square(np.subtract(y_train, p_train))
    train_error = train_error.mean()
    
    p_test = np.dot(X_test, w)
    test_error = np.square(np.subtract(y_test, p_test))
    test_error = test_error.mean()
    
    ##############
    return train_error, test_error

def k_fold_cross_validation(k, X, y):
    kf = KFold(n_splits=k, random_state=21, shuffle=True)
    total_E_val_test = 0
    total_E_val_train = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # determine the training error and the test error
        #### TO-DO #####
        w = get_coeff_linear(X_train, y_train)
        train_error, test_error = evaluate_err(X_train, X_test, y_train, y_test, w)
        total_E_val_train += train_error
        total_E_val_test += test_error
    
    E_val_train = total_E_val_train/k
    E_val_test = total_E_val_test/k
    
       ##############
    return  E_val_test, E_val_train

def score(w, X, y):
    y_pred = (X @ w).reshape(-1, )
    y.reshape(-1, )
    u = np.sum(np.square(y-y_pred))
    y_mean = np.full(y.shape, np.mean(y))
    v = np.sum(np.square(y-y_mean))
    return 1-(u/v)

MSE_test, MSE_train = k_fold_cross_validation(5, X, y)
print("5 crossfold validation")
print("MSE_test =", round(MSE_test, 3),  "\tMSE_train =", round(MSE_train,3))

print("\ntrain_test_split")
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.3,random_state=0)
w = get_coeff_linear(X_train, y_train)
# print(w)
train_error, test_error = evaluate_err(X_train, X_test, y_train, y_test, w)
print("linear regression train score:\t", score(w, X_train, y_train))
print("linear regression test score:\t", score(w, X_test, y_test))
print("MSE_test =", round(test_error, 3),  "\tMSE_train =", round(train_error,3))

5 crossfold validation
MSE_test = 3.634 	MSE_train = 3.388

train_test_split
linear regression train score:	 0.5820634460338829
linear regression test score:	 0.5644507133517508
MSE_test = 3.53 	MSE_train = 3.438


# Lasso Regression

In [3]:
# using coordinated descent to implement lasso regression
def get_coeff_lasso(X, y, alpha = .01, num_iters=100, intercept = False):
    m,n = X.shape
    
    # Initializing w to Ridge coefficients
    w = get_coeff_linear(X, y)
    
    aj = (2/m)*(np.sum(np.square(X), axis=0))
    
    for i in range(num_iters): 
        for j in range(n):
            wtx = (X @ w).reshape(-1,)
            X_j = X[:,j].reshape(-1,)
            a = (y - wtx).reshape(-1,)
            b = (w[j]*X_j).reshape(-1,)
            cj = (2/m)*(X_j @ (a+b))
        
            if j == 0 and intercept == True:
                w[j] =  w[j]
            else:
                if cj < -alpha:
                    w[j] = (cj + alpha)/aj[j]
                elif cj > alpha:
                    w[j] = (cj - alpha)/aj[j]
                else: 
                    w[j] = 0
    return w

def evaluate_err(X_train, X_test, y_train, y_test, w): 
    #### TO-DO #####
    p_train = np.dot(X_train, w)
    train_error = np.square(np.subtract(y_train, p_train))
    train_error = train_error.mean()
    
    p_test = np.dot(X_test, w)
    test_error = np.square(np.subtract(y_test, p_test))
    test_error = test_error.mean()
    
    ##############
    return train_error, test_error

def k_fold_cross_validation(k, X, y, alpha):
    kf = KFold(n_splits=k, random_state=21, shuffle=True)
    total_E_val_test = 0
    total_E_val_train = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
        #### TO-DO #####
        m,n = X_train.shape
        initial_w = np.ones((n,1))
        w = get_coeff_lasso(X_train, y_train, alpha, num_iters=1000, intercept=True)
        train_error, test_error = evaluate_err(X_train, X_test, y_train, y_test, w)
        total_E_val_train += train_error
        total_E_val_test += test_error
    
    E_val_train = total_E_val_train/k
    E_val_test = total_E_val_test/k
    
       ##############
    return  E_val_test, E_val_train

def score(w, X, y):
    y_pred = (X @ w).reshape(-1, )
    y.reshape(-1, )
    u = np.sum(np.square(y-y_pred))
    y_mean = np.full(y.shape, np.mean(y))
    v = np.sum(np.square(y-y_mean))
    return 1-(u/v)

alpha = .01
MSE_test, MSE_train = k_fold_cross_validation(5, X, y, alpha)
print("5 crossfold validation")
print("\u03BB =", alpha)
print("MSE_test =", round(MSE_test, 3),  "\tMSE_train =", round(MSE_train,3))

print("\ntrain_test_split")
print("\u03BB =", alpha)
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.3,random_state=0)
w = get_coeff_lasso(X_train, y_train, alpha, num_iters=1000, intercept=True)
train_error, test_error = evaluate_err(X_train, X_test, y_train, y_test, w)
print("lasso regression train score:\t", score(w, X_train, y_train))
print("lasso regression test score:\t", score(w, X_test, y_test))
print("MSE_test =", round(test_error, 3),  "\tMSE_train =", round(train_error,3))

5 crossfold validation
λ = 0.01
MSE_test = 3.632 	MSE_train = 3.408

train_test_split
λ = 0.01
lasso regression train score:	 0.5807017771843322
lasso regression test score:	 0.5669872636401383
MSE_test = 3.51 	MSE_train = 3.449
